In [6]:
import numpy as np, pandas as pd, random, matplotlib.pyplot as plt, seaborn as sns
from IPython.display import clear_output
from PyProbs import Probability as pr

class passenger(object):
    def __init__(self):
        self.aisle = 0
        self.seat = 0
        self.speed = pr.Prob(3/100)
        #self.purse = pr.Prob(50/100)
        #self.carry = pr.Prob(30/100)
        self.purse = 0
        self.carry = 0
        self.timer = 2
        self.first = 1
        self.action = 0
        self.board = 0

def create_passenger_list():
    p_list = []
    for i in range(32 * 6):
        p_list.append(passenger())
        p_list[i].aisle = i // 6
        p_list[i].seat = i % 6
    both = 0
    purse_o = 0
    carry_o = 0
    for i in range(192):
        a = pr.Prob(50 / 100)
        b = pr.Prob(30 / 100)
        if a and b:
            both += 1
        elif a:
            purse_o += 1
        elif b:
            carry_o += 1
    ind = 1
    while(both):
        both -= 1
        p_list[-ind].purse = 1
        p_list[-ind].carry = 1
        ind += 1
    while(carry_o):
        carry_o -= 1
        p_list[-ind].carry = 1
        ind += 1
    while(purse_o):
        purse_o -= 1
        p_list[-ind].purse = 1
        ind += 1
    p_list1 = []
    for i in range(32):
        p_list1.append(p_list[6*i])
    for i in range(32):
        p_list1.append(p_list[6*i + 5])
    for i in range(32):
        p_list1.append(p_list[6*i + 1])
    for i in range(32):
        p_list1.append(p_list[6*i + 4])
    for i in range(32):
        p_list1.append(p_list[6*i + 2])
    for i in range(32):
        p_list1.append(p_list[6*i + 3])
    p_list_2 = p_list1[:64]
    p_list_3 = p_list1[64:128]
    p_list_4 = p_list1[128:]
    random.shuffle(p_list_2)
    random.shuffle(p_list_3)
    random.shuffle(p_list_4)
    p_list1[:64] = p_list_2
    p_list1[64:128] = p_list_3
    p_list1[128:] = p_list_4
    p_list1.reverse()
    return p_list

def initialize_array(narrow):
    for i in range(32):
        narrow[i][7] = 12
    return narrow

def initialize_timer(p_list):
    for i in range(len(p_list)):
        p_list[i].timer += 3*p_list[i].purse + 5*p_list[i].carry
        if p_list[i].speed:
            p_list[i].speed = 49
        else:
            p_list[i].speed = 99
    return p_list

def plot_fig(narrow, viewer):
    for i in range(32):
        for j in range(7):
            if narrow[i][j] != 0:
                if j != 3:
                    viewer[i][j] = 2
                else:
                    if narrow[i][j].speed == 49:
                        viewer[i][j] = 7
                    else:
                        viewer[i][j] = 10
            else:
                viewer[i][j] = 0
    plt.close()
    fig, ax = plt.subplots()
    im = ax.imshow(viewer)
    plt.draw()
    plt.pause(0.0000001)

def run_narrow(p_list, narrow):
    plt.close("all")
    #fig, ax = plt.subplots()
    #plt.ion()
    #plt.show()
    sit = 0
    time = 0
    seated = []
    viewer = zeros = [[0] * 7 for _ in range(32)]
    while sit < 191:
        time += 1
        for i in range(32):
            change = 0
            if narrow[i][3] != 0 and narrow[i][3].action == 0:
                narrow[i][3].action = 1
                if narrow[i][3].aisle != i:
                    if narrow[i+1][3] == 0:
                        narrow[i][3].walk += 1
                        if random.randrange(100) < narrow[i][3].speed:
                            narrow[i+1][3] = narrow[i][3]
                            narrow[i][3] = 0
                            change = 1
                    else:
                        narrow[i][3].wait += 1
                else:
                    if narrow[i][3].first:
                        narrow[i][3].first = 0
                        narrow[i][7] -= narrow[i][3].purse + 2*narrow[i][3].carry
                        #if narrow[i][7] < 0:
                        #    narrow[i][3].timer += 5
                        if narrow[i][3].seat == 0:
                            narrow[i][3].timer += 5*narrow[i][1] + 3*narrow[i][2]
                        elif narrow[i][3].seat == 1:
                            narrow[i][3].timer += 3*narrow[i][2]
                        elif narrow[i][3].seat == 5:
                            narrow[i][3].timer += 3*narrow[i][4]
                        elif narrow[i][3].seat == 6:
                            narrow[i][3].timer += 5*narrow[i][5] + 3*narrow[i][4]
                        if narrow[i][3].speed == 49:
                            narrow[i][3].timer *= 2
                        narrow[i][3].bag = narrow[i][3].timer
                    if narrow[i][3].timer != 0:
                        narrow[i][3].timer -= 1
                    else:
                        change = 1
                        if narrow[i][3].seat >= 3:
                            narrow[i][narrow[i][3].seat + 1] = 1
                            sit += 1
                        else:
                            narrow[i][narrow[i][3].seat] = 1
                            sit += 1
                        seated.append(narrow[i][3])
                        narrow[i][3] = 0
            if i == 0:
                if narrow[i][3] == 0 and len(p_list) != 0:
                    narrow[i][3] = p_list.pop()
                    narrow[i][3].action = 1
                    change = 1
        plot_fig(narrow, viewer)

        for i in range(32):
            if narrow[i][3] != 0:
                narrow[i][3].action = 0
                narrow[i][3].board += 1
    board_time = []
    wait_time = []
    bag_time = []
    for i in range(len(seated)):
        board_time.append(seated[i].board)
        wait_time.append(seated[i].wait)
        bag_time.append(seated[i].bag)
    return time, np.mean(board_time), np.mean(wait_time), np.mean(bag_time)

time = []
board_avg = []
wait_avg = []
bag_avg = []
"""
for i in range(1000):
    p_list = create_passenger_list()
    p_list = initialize_timer(p_list)
    narrow = [[0] * 8 for _ in range(32)]
    narrow = initialize_array(narrow)
    a, b, c, d = run_narrow(p_list, narrow)
    time.append(a)
    board_avg.append(b)
    wait_avg.append(c)
    bag_avg.append(d)
print(np.mean(time), np.mean(board_avg))
np_time = np.array(time)
np_board_avg = np.array(board_avg)
np_wait_avg = np.array(wait_avg)
np_bag_avg = np.array(bag_avg)
np.save('our_time.npy', np_time)
np.save('our_board_avg.npy', np_board_avg)
np.save('our_wait_avg.npy', np_wait_avg)
np.save('our_bag_avg.npy', np_bag_avg)
"""
p_list = create_passenger_list()
for i in range(len(p_list)):
    print(p_list[i].aisle)

0
0
0
0
0
0
1
1
1
1
1
1
2
2
2
2
2
2
3
3
3
3
3
3
4
4
4
4
4
4
5
5
5
5
5
5
6
6
6
6
6
6
7
7
7
7
7
7
8
8
8
8
8
8
9
9
9
9
9
9
10
10
10
10
10
10
11
11
11
11
11
11
12
12
12
12
12
12
13
13
13
13
13
13
14
14
14
14
14
14
15
15
15
15
15
15
16
16
16
16
16
16
17
17
17
17
17
17
18
18
18
18
18
18
19
19
19
19
19
19
20
20
20
20
20
20
21
21
21
21
21
21
22
22
22
22
22
22
23
23
23
23
23
23
24
24
24
24
24
24
25
25
25
25
25
25
26
26
26
26
26
26
27
27
27
27
27
27
28
28
28
28
28
28
29
29
29
29
29
29
30
30
30
30
30
30
31
31
31
31
31
31
